In [177]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import math
import warnings
import tensorflow as tf
from tensorflow import keras
from matplotlib import pyplot
from scipy.stats import uniform, randint
warnings.filterwarnings('ignore')

# Model and performance evaluation
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from scikeras.wrappers import KerasClassifier
from joblib import dump, load
from sklearn.svm import SVC

from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score 
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import auc, average_precision_score, precision_recall_curve

# Hyperparameter tuninghCV
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV, RandomizedSearchCV
from hyperopt import tpe, STATUS_OK, Trials, hp, fmin, STATUS_OK, space_eval
from xgboost import plot_importance

# Data imputation
from imblearn.over_sampling import SMOTE


## 1. Functions

#### 1) Evaluate model

In [178]:
def get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    f1 = f1_score(y_test,pred)
    roc_auc = roc_auc_score(y_test, pred_proba)
    precision_, recall_, thr = precision_recall_curve(y_test, pred_proba)
    pr_auc = auc(recall_, precision_)
    result = '{0:.4f}, {1:.4f}, {2:.4f}, {3:.4f}, {4:.4f}, {5:.4f}'.format(accuracy, precision, recall, f1, roc_auc, pr_auc)
    return confusion, result 


#### 2) Plot AUROC curve

In [179]:
def roc_curve_plot(y_test, opted_predict_prob,label_name):
    fpr, tpr, thresholds = roc_curve(y_test, opted_predict_prob)

    plt.plot(fpr, tpr, label = label_name)
    plt.plot([0,1], 'k--')
    plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate')
    plt.title("AUROC Curves")
    plt.legend(fontsize=6)

#### 3) Plot AUPRC curve

In [180]:
def plot_pr_curve(recall, precision,label_name):
    pyplot.plot([0, 1], 'k--')
    pyplot.plot(recall, precision, label=label_name)
    # axis labels
    pyplot.xlabel('Recall')
    pyplot.ylabel('Precision')
    plt.title("AUPRC")
    pyplot.legend(fontsize=6)

#### 4) Preprocess_data

In [181]:
def preprocess_data(input, output):

    features = pd.read_excel(input)
    label = pd.read_excel(output)

    X_train, X_test, y_train, y_test = train_test_split(features, label, 
                                                    test_size=0.2, random_state=1)
    
    y_train = np.ravel(y_train)
    y_test = np.ravel(y_test)
    
    # imputating train_data
    smote = SMOTE(random_state=0)
    X_train, y_train = smote.fit_resample(X_train, y_train)
    
    return X_train, X_test, y_train, y_test

#### 5) Plot_curve
version 1과 version 2 중에 하나만 실행하기. 
실행 후 각 모델별 함수('## Ploting curve' 확인)에서 실행될 수 있도록 주석 확인.

In [182]:
# version 1_input 또는 output 고정하여 비교할 경우 실행
def plot_curve(input, output, curve, y_test, opted_predict_prob):
    
    # set label
    in_ = re.sub(r'[^0-9]', '', input) 
    out_ = re.sub(r'[^0-9]', '', output)
    in_ = int(in_)
      
    # plot AUCROC curve
    if curve == 'auroc':
        roc_auc = round(roc_auc_score(y_test, opted_predict_prob), 3)
        label = f"{in_-2}~{in_} Hours after ICU admission, {curve.upper()}: {roc_auc}"
        roc_curve_plot(y_test, opted_predict_prob, label_name = label)
        
    # plot AUPRC curve
    elif curve == 'auprc':
        precision, recall, thresholds = precision_recall_curve(y_test, opted_predict_prob)
        pr_auc = round(auc(recall, precision),3)
        label = f"{in_-2}~{in_} Hours after ICU admission, {curve.upper()}: {pr_auc}"
        plot_pr_curve(recall, precision, label_name=label)

In [183]:
# version 2_MODEL별로 비교할 경우 실행
def plot_curve_(input, output, curve, y_test, opted_predict_prob, model_name):
    
    # set label
    in_ = re.sub(r'[^0-9]', '', input) 
    out_ = re.sub(r'[^0-9]', '', output)
      
    # plot AUROC curve
    if curve == 'auroc':
        roc_auc = round(roc_auc_score(y_test, opted_predict_prob), 3)
        label = f"{model_name}, {curve.upper()}: {roc_auc}"
        roc_curve_plot(y_test, opted_predict_prob, label_name = label)
        
    # plot AUPRC curve
    elif curve == 'auprc':
        precision, recall, thresholds = precision_recall_curve(y_test, opted_predict_prob)
        pr_auc = round(auc(recall, precision),3)
        label = f"{model_name}, {curve.upper()}: {pr_auc}"
        plot_pr_curve(recall, precision, label_name=label)

#### 6) Train & Calibrate

In [184]:
def train_and_calibrate_model(name, model, calibration, param, X_train, y_train, X_test, y_test, input, output):    
    in_ = re.sub(r'[^0-9]', '', input)
    out_ = re.sub(r'[^0-9]', '', output)

    # Grid search CV
    if calibration == 'grid' :
        
        scoring = ['roc_auc']
   
        kfold = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=1)

        grid_search = GridSearchCV(estimator=model, 
                           param_grid=param, 
                           scoring= scoring, 
                           refit='roc_auc', 
                           n_jobs=-1, 
                           cv=kfold, 
                           verbose=0)
    
        grid_result = grid_search.fit(X_train, y_train)
    
        model = grid_search.best_estimator_

        opted_predict = grid_search.predict(X_test)
    
        # Get predicted probabilities
        opted_predict_prob = grid_search.predict_proba(X_test)[:,1]
  
    # Randomized search CV
    elif calibration == 'random':

        scoring = ['roc_auc']
  
        kfold = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=0)

        random_search = RandomizedSearchCV(estimator=model, 
                           param_distributions=param, 
                           n_iter=48,
                           scoring=scoring, 
                           refit='roc_auc', 
                           n_jobs=-1, 
                           cv=kfold, 
                           verbose=0)
  
        random_result = random_search.fit(X_train, y_train)
    
        dump(random_search, f'./trained/{name}_matrix_trained_{calibration }_{in_}hr_{out_}hr.joblib') 
        print(f"save matrix: {name}_matrix_trained_{calibration }_{in_}hr_{out_}hr.joblib")

        opted_predict = random_search.predict(X_test)
        opted_predict_prob = random_search.predict_proba(X_test)[:,1]
    
    return opted_predict, opted_predict_prob

## 2. Models

#### 1) ANN

In [201]:
def ANN_model(input, output, n_fold, calibration, curve):
    
    ## processing data for training/test
    features = pd.read_excel(input)
    label = pd.read_excel(output)
    
    len_features = int(features.shape[1])

    X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.2, random_state=42)
    
    y_train = np.ravel(y_train)
    y_test = np.ravel(y_test)
    
    smote = SMOTE(random_state=0)
    X_train, y_train = smote.fit_resample(X_train, y_train)

    
    ## model 
    model = keras.Sequential()
    
    model.add(keras.layers.InputLayer(input_shape=(len_features,)))
    
    model.add(keras.layers.Dense(10, activation='softmax'))

    model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    model = KerasClassifier(model)

    ## optimizing hyper-param for model
    
    # # Grid search CV
    if calibration == 'grid' :
        param = {'batch_size': [16, 32, 64, 128],
                 'epochs': [10, 20]
                }

        grid_search = GridSearchCV(estimator=model, param_grid=param, scoring='accuracy', cv=3)

        grid_result = grid_search.fit(X_train, y_train)

        best_model = grid_search.best_estimator_
        opted_predict = best_model.predict(X_test)
        
        opted_predict_prob = grid_search.predict_proba(X_test)[:,1]
    
        
  
    # Randomized search CV
    elif calibration == 'random':
        param = {'batch_size': [16, 32, 64, 128],
                 'epochs': [10, 20]
                }
        
         # Set up score
        scoring = ['accuracy']
   
        kfold = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=1)


        random_search = RandomizedSearchCV(estimator=model, 
                                   param_distributions=param, 
                                   scoring=scoring, 
                                   cv=kfold,
                                   refit='accuracy', 
                                   n_jobs=-1, 
                                   verbose=0)

        random_result = random_search.fit(X_train, y_train)

        best_model = random_search.best_estimator_
        opted_predict = best_model.predict(X_test)
        
        opted_predict_prob = random_search.predict_proba(X_test)[:,1]
    
    ## Ploting curve
    plot_curve(input, output, curve, y_test, opted_predict_prob)
    
    ## Get performance metrics
    temp_comfusion, temp_result = get_clf_eval(y_test , opted_predict, opted_predict_prob)

    return temp_comfusion, temp_result


#### 2) Decision Tree

In [185]:
def LR_model(name, input, output, n_fold, calibration, curve):
    
    X_train, X_test, y_train, y_test = preprocess_data(input, output)

    
    ## model 
    model = LogisticRegression(random_state=1, max_iter=2000)

    ## optimizing hyper-param for model 
    if calibration == 'grid' :
        param = {'C':[0.001,0.01,0.1,1,10], 
                 'penalty':['l1', 'l2']  
                }
    elif calibration == 'random':
        
        param = {'C':[0.001,0.01,0.1,1,10],
                 'penalty': ['l1','l2'],
                }
        
    opted_predict, opted_predict_prob = train_and_calibrate_model(name, model, calibration, param, X_train, y_train, X_test, y_test, input, output)
    
    ## Ploting curve
    plot_curve(input, output, curve, y_test, opted_predict_prob)
    
    ## Get performance metrics
    temp_comfusion, temp_result = get_clf_eval(y_test , opted_predict, opted_predict_prob)

    return temp_comfusion, temp_result


#### 3) KNN

In [188]:
def KNN_model(name, input, output, n_fold, calibration, curve):
    
    X_train, X_test, y_train, y_test = preprocess_data(input, output)
    
    ## model 
    model = KNeighborsClassifier(n_neighbors=5)

    ## optimizing hyper-param for model 
    if calibration == 'grid' :
        param = {'n_neighbors' : list(range(1,30)),
                 'weights' : ["uniform", "distance"],
                 'metric' : ['euclidean', 'manhattan', 'minkowski'],
                 'leaf_size' : range(1, 50, 5)
                }
        
    elif calibration == 'random':
  
        param = {'n_neighbors' : list(range(1,30)),
                 'weights' : ["uniform", "distance"],
                 'metric' : ['euclidean', 'manhattan', 'minkowski'],
                 'leaf_size' : range(1, 50, 5)
                }

    opted_predict, opted_predict_prob = train_and_calibrate_model(name, model, calibration, param, X_train, y_train, X_test, y_test, input, output)
    
    ## Ploting curve
    plot_curve(input, output, curve, y_test, opted_predict_prob)
    
    ## Get performance metrics
    temp_comfusion, temp_result = get_clf_eval(y_test , opted_predict, opted_predict_prob)

    return temp_comfusion, temp_result


#### 4) Logistic Regeression

In [186]:
def DT_model(name, input, output, n_fold, calibration, curve):
    
    X_train, X_test, y_train, y_test = preprocess_data(input, output)
    
    ## model 
    model = DecisionTreeClassifier(random_state=1)

    ## optimizing hyper-param for model
    if calibration == 'grid' :
        param = {'min_impurity_decrease': [0.001, 0.01, 0.1],
                 'max_depth': range(5, 20, 1),
                 'min_samples_split': range(2, 100, 10)
                }
    elif calibration == 'random':
  
        param = {'min_impurity_decrease': [0.001, 0.01, 0.1],
                 'max_depth': range(5, 20, 1),
                 'min_samples_split': range(2, 100, 10)
                }

    opted_predict, opted_predict_prob = train_and_calibrate_model(name, model, calibration, param, X_train, y_train, X_test, y_test, input, output)
    
    ## Ploting curve
    plot_curve(input, output, curve, y_test, opted_predict_prob)
    
    ## Get performance metrics
    temp_comfusion, temp_result = get_clf_eval(y_test , opted_predict, opted_predict_prob)

    return temp_comfusion, temp_result


#### 5) Random Forest

In [187]:
def RF_model(name, input, output, n_fold, calibration, curve):
    
    X_train, X_test, y_train, y_test = preprocess_data(input, output)
    
    ## model 
    model = RandomForestClassifier(n_jobs=-1, random_state=1)

    ## optimizing hyper-param for model
    if calibration == 'grid' :
    
        param = {'n_estimators' : [10, 100],
                 'max_depth' : [6, 8, 10, 12],
                 'min_samples_leaf' : [8, 12, 16, 20],
                 'min_samples_split' : [8, 12, 16, 20]
                }
        
    elif calibration == 'random':
  
        param = {'n_estimators' : [10, 100],
                 'max_depth' : [6, 8, 10, 12],
                 'min_samples_leaf' : [8, 12, 16, 20],
                 'min_samples_split' : [8, 12, 16, 20]
                }

    opted_predict, opted_predict_prob = train_and_calibrate_model(name, model, calibration, param, X_train, y_train, X_test, y_test, input, output)
    
    ## Ploting curve
    plot_curve(input, output, curve, y_test, opted_predict_prob)
    
    ## Get performance metrics
    temp_comfusion, temp_result = get_clf_eval(y_test , opted_predict, opted_predict_prob)

    return temp_comfusion, temp_result


#### 6) SVM

In [191]:
def SVM_model(name, input, output, n_fold, calibration, curve):
    
    ## processing data for training/test
    X_train, X_test, y_train, y_test = preprocess_data(input, output)

    
    ## model 
    model = SVC(random_state=0, probability=True)

    ## optimizing hyper-param for model
    
    # grid search 
    if calibration == 'grid' :
        param = {'C': [0.1,1, 10, 100], 
         'kernel': ['poly','sigmoid','linear','rbf'],
         'gamma': [1,0.1,0.01,0.001]}

    # random search
    elif calibration == 'random':
  
        param = {'C': [0.1,1, 10, 100], 
         'kernel': ['poly','sigmoid','linear','rbf'],
         'gamma': [1,0.1,0.01,0.001]}
    
    opted_predict, opted_predict_prob = train_and_calibrate_model(name, model, calibration, param, X_train, y_train, X_test, y_test, input, output)

    ## Ploting curve
    plot_curve(input, output, curve, y_test, opted_predict_prob)
    
    ## Get performance metrics
    temp_comfusion, temp_result = get_clf_eval(y_test , opted_predict, opted_predict_prob)


    return temp_comfusion, temp_result

#### 7) XGBoost

In [189]:
def XGB_model(name, input, output, n_fold, calibration, curve):
    
    X_train, X_test, y_train, y_test = preprocess_data(input, output)
    
    ## model 
    model = XGBClassifier(booster='gbtree', objective='binary:logistic')

    ## optimizing hyper-param for model
    if calibration == 'grid' :
        param = {'n_estimators': [100,200,400,800,1000],
                 'learning_rate': [0.01,0.05,0.1,0.2,0.3,0.4,0.5],
                 'gamma': [0,0.01,0.1,0.5,1,2],
                 'min_child_weight':[1,2,3,4,5]
                }
    # random search
    elif calibration == 'random':
  
        param = {'n_estimators': [100,200,400,800,1000],
                 'learning_rate': [0.01,0.05,0.1,0.2,0.3,0.4,0.5],
                 'gamma': [0,0.01,0.1,0.5,1,2],
                 'min_child_weight':[1,2,3,4,5]
                }

    opted_predict, opted_predict_prob = train_and_calibrate_model(name, model, calibration, param, X_train, y_train, X_test, y_test, input, output)
    
    ## Ploting curve
    plot_curve(input, output, curve, y_test, opted_predict_prob)
    
    ## Get performance metrics
    temp_comfusion, temp_result = get_clf_eval(y_test , opted_predict, opted_predict_prob)

    return temp_comfusion, temp_result


## 3. Execute model

아래 세 함수 중 하나만 실행하기.

#### 1) Input 고정

In [37]:
def execute(dir_input, dir_output, n_fold, imputation, calibration, curve, name):    
    i = 0
    total = len(dir_input)*len(dir_output)
    df_result = pd.DataFrame(columns = ["Accuracy", "Precision", "Recall", "F1", "AUROC", "AUPRC"])
    
    models = {'ANN': ANN_model,
              'DT': DT_model,
              'KNN': KNN_model,
              'LR': LR_model,
              'RF': RF_model,
              'SVM': SVM_model,
              'XGB': XGB_model
             }
    
    model = models[name]
    
    for input in dir_input:
        in_ = re.sub(r'[^0-9]', '', input)
        idx = {'Accuracy': f'{in_}hr'}
        df_result = df_result.append(idx, ignore_index=True)
  
        for output in dir_output:
            print(f"{i/total*100}% processing..")
        
            ## Run
            confusion, result = model(name, input, output, n_fold, calibration, curve)
        
            ## post-process results from model
            dic = {}
            temp = result.split(', ')

            index = 0
            for key in df_result.keys():
                dic[key] = temp[index]
                index += 1

            df_result = df_result.append(dic, ignore_index=True)
            plt.savefig(f'./result_graph/{name}_{imputation}_{curve}_curve_{out_}hr.png', dpi=300)
            i = i + 1
        
        plt.cla()

    # export    
    df_result.to_csv(f'./result_csv/result_{name}_{calibration}_{imputation}_prolonged.csv')

#### 2) Output 고정

In [202]:
def execute(dir_input, dir_output, n_fold, imputation, calibration, curve, name):    
    i = 0
    total = len(dir_input)*len(dir_output)
    df_result = pd.DataFrame(columns = ["Accuracy", "Precision", "Recall", "F1", "AUROC", "AUPRC"])
    
    models = {'ANN': ANN_model,
              'DT': DT_model,
              'KNN': KNN_model,
              'LR': LR_model,
              'RF': RF_model,
              'SVM': SVM_model,
              'XGB': XGB_model
             }
    
    model = models[name]
    
    for output in dir_output:
        out_ = re.sub(r'[^0-9]', '', output)
        idx = {'Accuracy': f'{out_}hr'}
        df_result = df_result.append(idx, ignore_index=True)
    
        for input in dir_input:
            
            in_ = re.sub(r'[^0-9]', '', input)
            print(f"{i/total*100}% processing..")
        
            ## Run
            confusion, result = model(name, input, output, n_fold, calibration, curve)
        
            ## post-process results from model
            dic = {}
            temp = result.split(', ')

            index = 0
            for key in df_result.keys():
                dic[key] = temp[index]
                index += 1

            df_result = df_result.append(dic, ignore_index=True)
            plt.savefig(f'./result_graph/{name}_{imputation}_{curve}_curve_{out_}hr.png', dpi=300)
            i = i + 1
        
        plt.cla()
        
        # export    
        df_result.to_csv(f'./result_csv/result_{name}_{calibration}_{imputation}_prolonged.csv')


#### 3) 모델별 그래프 (input, output 고정)

In [170]:
def execute(input_path, output_path, n_fold, imputation, calibration, curve, list_models):    
     
    models = {'ANN': ANN_model,
              'DT': DT_model,
              'KNN': KNN_model,
              'LR': LR_model,
              'RF': RF_model,
              'SVM': SVM_model,
              'XGB': XGB_model
             }
    
    
    in_ = re.sub(r'[^0-9]', '', input_path)
    out_ = re.sub(r'[^0-9]', '', output_path)
    
    ANN_model(input_path, output_path, 'random', curve)
        
    plt.savefig(f'./result_graph/ALL_{imputation}_{curve}_curve_{in_}hr.png', dpi=300)
    
    total = len(list_models)
    i = 0
    
    for name in list_models:
        if name == 'ANN':
            break
        
        model = models[name]
        
        print(f"{i/total*100}% processing..")
        
        ## Run
        confusion, result = model(name, input_path, output_path, n_fold, calibration, curve)
    
        plt.savefig(f'./result_graph/ALL_{imputation}_{curve}_curve_{in_}hr_{out_}hr.png', dpi=300)
        
        i = i + 1
        
        


#### 4) Configure

In [203]:

dir_input = [
    './df_input/df_2hr.xlsx',
    './df_input/df_4hr.xlsx',
    './df_input/df_6hr.xlsx',
    './df_input/df_8hr.xlsx',
    './df_input/df_10hr.xlsx',
    './df_input/df_12hr.xlsx'
]

dir_output = [
      './df_output/df_categorical_output_hr/df_categorical_output_24hr.xlsx',
     './df_output/df_categorical_output_hr/df_categorical_output_36hr.xlsx',
    './df_output/df_categorical_output_hr/df_categorical_output_48hr.xlsx',
    './df_output/df_categorical_output_hr/df_categorical_output_60hr.xlsx',
    './df_output/df_categorical_output_hr/df_categorical_output_72hr.xlsx',
    './df_output/df_categorical_output_hr/df_categorical_output_120hr.xlsx',
    './df_output/df_categorical_output_hr/df_categorical_output_168hr.xlsx',
    './df_output/df_categorical_output_hr/df_categorical_output_336hr.xlsx'
 ]  

n_fold = 5
imputation = 'smote'
calibration = 'random'
curves = ['auroc'] # auroc: AUROC/ auprc: AUPRC

In [ ]:
# 모델별로 성능을 비교할 경우에 사용

dir_input = './df_input/df_12hr.xlsx'
dir_output = './df_output/df_categorical_output_hr/df_categorical_output_60hr.xlsx'

models = ['ANN','DT','KNN','LR','RF','SVM','XGB']
n_fold = 5
imputation = 'smote'
calibration = 'random'
curves = ['auroc', 'auprc'] # auroc: AUROC/ auprc: AUPRC

for curve in curves:
    execute(dir_input, dir_output, n_fold, imputation, calibration, curve, models)
    plt.cla()


#### 5) Execute single model

In [ ]:
model = ['SVM'] #'LR','DT','RF','XGB','KNN','SVM'
curves = ['auroc', 'auprc'] # auroc: AUROC/ auprc: AUPRC

for curve in curves:
    execute(dir_input, dir_output, n_fold, imputation, calibration, curve, model)
    plt.cla()

In [ ]:
## ANN single execution

curves = ['auroc', 'auprc'] # auroc: AUROC/ auprc: AUPRC

for curve in curves:
    i = 0
    total = len(dir_input)*len(dir_output)
    df_result = pd.DataFrame(columns = ["Accuracy", "Precision", "Recall", "F1", "ROC AUC", "PR AUC"])
    
    for output in dir_output:
        out_ = re.sub(r'[^0-9]', '', output)
        idx = {'Accuracy': f'{out_}hr'}
        df_result = df_result.append(idx, ignore_index=True)
    
        for input in dir_input:
            
            in_ = re.sub(r'[^0-9]', '', input)
            print(f"{i/total*100}% processing..")

        
            ## Run
            confusion, result = ANN_model(input, output, n_fold, calibration, curve)
        
            ## post-process results from model
            dic = {}
            temp = result.split(', ')

            index = 0
            for key in df_result.keys():
                dic[key] = temp[index]
                index += 1

            df_result = df_result.append(dic, ignore_index=True)
            plt.savefig(f'./result_graph/ANN_{imputation}_{curve}_curve_{out_}hr.png', dpi=300)
            i = i + 1
        
        plt.cla()

    # export    
    df_result.to_csv(f'./result_csv/result_ANN_{calibration}_{imputation}_prolonged.csv')

#### 6) Execute multiple models

In [ ]:
models = ['ANN','DT','KNN','LR','RF','SVM','XGB']
curves = ['auroc', 'auprc'] # auroc: AUROC/ auprc: AUPRC

for model in models:
    for curve in curves:
        execute(dir_input, dir_output, n_fold, imputation, calibration, curve, model)